In [18]:
import cv2
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import pickle

In [12]:
import sys
sys.path.insert(1, '../commons')
from features_extraction import *

# Set param

In [13]:
BLURRED=False

# Read data

In [14]:
img_full = cv2.imread('full_image.jpg')
img_bkg = cv2.imread('train&test data/background.jpg')
img_pallet = cv2.imread('train&test data/pallet.jpg')

if BLURRED:
    blur_size = 7
    img_full = cv2.medianBlur(img_full, blur_size)
    img_bkg = cv2.medianBlur(img_bkg, blur_size)
    img_pallet = cv2.medianBlur(img_pallet, blur_size)

# Extract features and split data

In [15]:
channels = get_chanels()

img_bkg_features = features(img_bkg, channels)
img_pallet_features = features(img_pallet, channels)

X = np.concatenate((img_bkg_features, img_pallet_features))
y = np.concatenate(([0]*len(img_bkg_features), [1]*len(img_pallet_features)))

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)
      
train_index, test_index = sss.split(X, y).__next__()
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

# Fit, save and test

In [19]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
prefix = "blurred_" if BLURRED else ""
filename = prefix+'naive_bayes_model.sav'
pickle.dump(gnb, open(filename, 'wb'))

y_pred=gnb.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[722551, 232943],
       [185436, 333153]])